<a href="https://colab.research.google.com/github/Hossein-Fallahi/AML-survival-analysis/blob/main/AML_ResistantResponse16062022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# One of the challenges in curing cancer is using chemothrapy for treatment. However, not all patients benefite from all types of therapy. 
#Here, we rae trying to use ML to see which group of patient repond to what type of therpy. 

## Data and environment

In [ ]:
# First we need to get the required libraries for starting the project. Later we migth need more libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load the data from your machine- You can modify the code to do this in googlecolab. 
from google.colab import files
uploaded = files.upload()
df = pd.read_csv('aml-rppa.csv')

# the data is in a dataframe. look at the head of the data
df.head(7)

# Data manipulation

In [ ]:
 #Lets look a bit dipper to the data- starting with the count of the number of rows and columns
df.shape 

In [ ]:
# Some coulmns contain un necessary information- drop all non essential columns for our analysis
df.drop(df.columns[[0, 1, 2,6,7,14]], axis = 1, inplace = True)
#note that eachtime the df is getting updated! probably i could do much better to drop all in one go!
df.drop(df.columns[[0,1,2,4,5,6,7]], axis = 1, inplace = True)
df.drop(df.columns[[0,1,2]], axis = 1, inplace = True)
df.drop(df.columns[[0,1,2,3,4,5,6]], axis = 1, inplace = True)
df.drop(df.columns[[1,3,5,6]], axis = 1, inplace = True)

df

In [ ]:
#count of the number of empty values (e.g. NaN, NAN, na) in each column
df.isna().sum()

In [ ]:
# drop the coulmns with empty values. 
df.drop(df.columns[[0]], axis = 1, inplace = True)
df

In [ ]:
# OS migth not be useful here. drop this coulmn as well
df.drop(df.columns[[0]], axis = 1, inplace = True)
df

In [ ]:
#drop the columns with all missing values- I did not drop the NA containg couumns as Response coulmn would be removed
#df = df.dropna(axis=1)

In [ ]:
#get the new coumnt of the data
df.shape

In [ ]:
#For the response coulmn, get the count of CR, Resistance or FAIL occurances
df['Response'].value_counts()


In [ ]:
# visualize the counts in a graph
sns.countplot(df['Response'], label = 'count')

In [ ]:
#look at the data type to see whcih coulmn need to be transformed (encoded)- 
df.dtypes

In [ ]:
#get the column index for "response"
df.columns.get_loc("Response")
df.iloc[:,0].values

In [ ]:
# encode the categorical datavalues
from sklearn.preprocessing import LabelEncoder
labelencoder_Y = LabelEncoder()
df.iloc[:,0] = labelencoder_Y.fit_transform(df.iloc[:,0].values)
df.iloc[:,0]
#get ride of 0's as they are decoded for "nan" values
df = df[df.Response != 0]


In [ ]:
df

In [ ]:
#get ride of CR's, which are coded as 3- The remaing are Fails or resistance
df = df[df.Response != 3]
df

In [ ]:
# creat a pair plot- not for all only the 5 first coulmns
sns.pairplot(df.iloc[:,0:6], hue ='Response')

In [ ]:
# print the first 5 row of the new dataset
df.head(5)

In [ ]:
#get the correlation of the coulmns
df.iloc[:,0:12].corr()

In [ ]:
#visualize the correlation
plt.figure(figsize=(10,10))
sns.heatmap(df.iloc[:,0:12].corr(), annot=True, fmt ='.0%')

In [ ]:
# some additonal data manipulation- I migth do this a littel eralier
df = df.replace(['NO'],'0.0')
df = df.replace(['YES'],'1.0')
#remove column with NaN values- 
df = df.dropna(axis=1)
df


# Spliting data and modelling

In [ ]:
#Split the dataset into independent (X) and dependet (Y)
X = df.iloc[:,1:].values
Y = df.iloc[:,0].values

#type(X) it is an array

In [ ]:
#split 75% training and test 25%
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y ,test_size = 0.25, random_state = 0)

In [ ]:
# Scale the data (Feature Scaling)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [ ]:
#creat a few models using a function- here we try 3 models
def models (X_train, Y_train):

  #logistic regression
  from sklearn.linear_model import LogisticRegression
  log = LogisticRegression(random_state =0)
  log.fit(X_train, Y_train)

  #Decision Tree
  from sklearn.tree import DecisionTreeClassifier
  tree = DecisionTreeClassifier(criterion = 'entropy', random_state=0)
  tree.fit(X_train, Y_train)

  #Random forest classifier
  from sklearn.ensemble import RandomForestClassifier
  forest = RandomForestClassifier(n_estimators = 10 , criterion= 'entropy', random_state = 0)
  forest.fit(X_train, Y_train)

  #print the model accuracy
  print('[0]Logistic regression accuracy :', log.score(X_train, Y_train))
  print('[1]Decision Tree accuracy :', tree.score(X_train, Y_train))
  print('[2]Random Forest accuracy :', forest.score(X_train, Y_train))

  return log, tree, forest

In [ ]:
#use the function to get the accuracy 
model = models (X_train, Y_train)

In [ ]:
#test the model on test data confusion matrix
from sklearn.metrics import confusion_matrix
for i in range(len(model)):
    print('Model ', i)
    cm = confusion_matrix(Y_test, model[i].predict(X_test))

    TP = cm[0][0]
    TN = cm[1][1]
    FN = cm[1][0]
    FP = cm[0][1]

    print(cm)
    print('testing accuracy = ', (TP + TN)/ (TP + TN + FN + FP))
    print()

In [ ]:
#show anothoer way to get matrix for the accuracy
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

for i in range(len(model)):
    print('Model ', i)
    print(classification_report(Y_test, model[i].predict(X_test)))
    print(accuracy_score(Y_test, model[i].predict(X_test)))
    print()

In [ ]:
#See how the model performed. e.g, preditcion of random forest model
pred = model[2].predict(X_test)
print(pred)
print()
print(Y_test)


As you can see the model performance is not that great. Work on data sources to find better predictors with higher ranking values. 